import pandas as pd

JejuData_train = pd.read_csv('C:/Users/82108/JejuRoad/JejuRoadData/open/train.csv')
JejuData_test = pd.read_csv('C:/Users/82108/JejuRoad/JejuRoadData/open/test.csv')
JejuData_sb = pd.read_csv('C:/Users/82108/JejuRoad/JejuRoadData/open/sample_submission.csv')

JejuData_train = JejuData_train.drop(['road_name','start_node_name','end_node_name'],axis=1)

JejuData_train['road_type']

test_1=JejuData_train[JejuData_train['connect_code'] == 103]

test_1.to_csv("test_1.csv", mode='w')

In [30]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import gc

In [2]:
from sklearn.datasets import load_boston
boston = load_boston()

In [3]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [4]:
from datetime import datetime
import requests
import json
import pandas as pd
from pandas import json_normalize

In [5]:
today = datetime.today().strftime('%Y%m%d')
today_year = 2020
    
key = 'UhnjyvmWEq5SH4yCqPylCPQSp4NdA524up%2FM1CrsUf3143Vmj3Rwx7PGHxq5YZY4mFHNHQ%2F0ixGwapRFApPsaw%3D%3D'
url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getRestDeInfo?_type=json&numOfRows=50&solYear=' + str(today_year) + '&ServiceKey=' + str(key)
response = requests.get(url)
if response.status_code == 200:
	json_ob = json.loads(response.text)
	holidays_data = json_ob['response']['body']['items']['item']
	dataframe = json_normalize(holidays_data)
dateName = dataframe.loc[dataframe['locdate'] == int(today), 'dateName']

print(dataframe)

   dateKind     dateName isHoliday   locdate  seq
0        01         1월1일         Y  20200101    1
1        01           설날         Y  20200124    1
2        01           설날         Y  20200125    1
3        01           설날         Y  20200126    1
4        01           설날         Y  20200127    1
5        01          삼일절         Y  20200301    1
6        01  제21대 국회의원선거         Y  20200415    1
7        01       부처님오신날         Y  20200430    1
8        01         어린이날         Y  20200505    1
9        01          현충일         Y  20200606    1
10       01          광복절         Y  20200815    1
11       01        임시공휴일         Y  20200817    1
12       01           추석         Y  20200930    1
13       01           추석         Y  20201001    1
14       01           추석         Y  20201002    1
15       01          개천절         Y  20201003    1
16       01          한글날         Y  20201009    1
17       01        기독탄신일         Y  20201225    1


In [6]:
def csv_to_parquet(csv_path, save_name):
    df = pd.read_csv(csv_path)
    df.to_parquet(f'./{save_name}.parquet')
    del df
    gc.collect()
    print(save_name, 'Done.')

In [7]:
csv_to_parquet('C:/Users/82108/JejuRoad/JejuRoadData/open/train.csv', 'train')
csv_to_parquet('C:/Users/82108/JejuRoad/JejuRoadData/open/test.csv', 'test')

train Done.
test Done.


In [8]:
train = pd.read_parquet('./train.parquet')
test = pd.read_parquet('./test.parquet')

In [9]:
min(train['base_date'])

20210901

In [10]:
train

,id,base_date,day_of_week,base_hour,lane_count,road_rating,road_name,multi_linked,connect_code,maximum_speed_limit,...,road_type,start_node_name,start_latitude,start_longitude,start_turn_restricted,end_node_name,end_latitude,end_longitude,end_turn_restricted,target
0,TRAIN_0000000,20220623,목,17,1,106,지방도1112호선,0,0,60.0,...,3,제3교래교,33.427747,126.662612,없음,제3교래교,33.427749,126.662335,없음,52.0
1,TRAIN_0000001,20220728,목,21,2,103,일반국도11호선,0,0,60.0,...,0,광양사거리,33.500730,126.529107,있음,KAL사거리,33.504811,126.526240,없음,30.0
2,TRAIN_0000002,20211010,일,7,2,103,일반국도16호선,0,0,80.0,...,0,창고천교,33.279145,126.368598,없음,상창육교,33.280072,126.362147,없음,61.0
3,TRAIN_0000003,20220311,금,13,2,107,태평로,0,0,50.0,...,0,남양리조트,33.246081,126.567204,없음,서현주택,33.245565,126.566228,없음,20.0
4,TRAIN_0000004,20211005,화,8,2,103,일반국도12호선,0,0,80.0,...,0,애월샷시,33.462214,126.326551,없음,애월입구,33.462677,126.330152,없음,38.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4701212,TRAIN_4701212,20211104,목,16,1,107,-,0,0,50.0,...,0,대림사거리,33.422145,126.278125,없음,금덕해운,33.420955,126.273750,없음,20.0
4701213,TRAIN_4701213,20220331,목,2,2,107,-,0,0,80.0,...,3,광삼교,33.472505,126.424368,없음,광삼교,33.472525,126.424890,없음,65.0
4701214,TRAIN_4701214,20220613,월,22,2,103,일반국도12호선,0,0,60.0,...,0,고성교차로,33.447183,126.912579,없음,성산교차로,33.444121,126.912948,없음,30.0
4701215,TRAIN_4701215,20211020,수,2,2,103,일반국도95호선,0,0,80.0,...,0,제6광령교,33.443596,126.431817,없음,관광대학입구,33.444996,126.433332,없음,73.0


In [11]:
test

,id,base_date,day_of_week,base_hour,lane_count,road_rating,road_name,multi_linked,connect_code,maximum_speed_limit,...,height_restricted,road_type,start_node_name,start_latitude,start_longitude,start_turn_restricted,end_node_name,end_latitude,end_longitude,end_turn_restricted
0,TEST_000000,20220825,목,17,3,107,연삼로,0,0,70.0,...,0.0,0,산지2교,33.499427,126.541298,없음,제주은행사거리,33.500772,126.543837,있음
1,TEST_000001,20220809,화,12,2,103,일반국도12호선,0,0,70.0,...,0.0,3,중문입구,33.258507,126.427003,없음,관광단지입구,33.258119,126.415840,없음
2,TEST_000002,20220805,금,2,1,103,일반국도16호선,0,0,60.0,...,0.0,0,도순3교,33.258960,126.476508,없음,도순2교,33.259206,126.474687,없음
3,TEST_000003,20220818,목,23,3,103,일반국도11호선,0,0,70.0,...,0.0,0,아라주공아파트,33.473494,126.545647,없음,인다마을,33.471061,126.545467,없음
4,TEST_000004,20220810,수,17,3,106,번영로,0,0,70.0,...,0.0,0,부록교 시종점,33.501477,126.569223,없음,봉개교 시종점,33.496863,126.581230,없음
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291236,TEST_291236,20220827,토,5,1,103,일반국도16호선,0,0,80.0,...,0.0,0,월산마을,33.472819,126.463030,없음,도근교,33.470483,126.460846,없음
291237,TEST_291237,20220819,금,20,2,103,일반국도11호선,0,0,60.0,...,0.0,0,영주교,33.305359,126.598914,없음,하례입구삼거리,33.300796,126.600332,없음
291238,TEST_291238,20220805,금,11,1,107,신대로,0,0,30.0,...,0.0,0,마리나사거리,33.493624,126.496769,있음,칠오광장사거리,33.497500,126.496946,있음
291239,TEST_291239,20220812,금,7,2,107,경찰로,0,0,60.0,...,0.0,0,국민의료보험관리공단,33.254782,126.507014,없음,의료보험조합,33.255659,126.507333,없음


In [14]:
str_col = ['start_turn_restricted','end_turn_restricted']
for i in str_col:
    le = LabelEncoder()
    le=le.fit(train[i])
    train[i]=le.transform(train[i])
    
    for label in np.unique(test[i]):
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test[i]=le.transform(test[i])

In [15]:
y_train = train['target'] 

X_train = train.drop(['id','base_date', 'target','road_name', 'start_node_name', 'end_node_name','vehicle_restricted','road_type','weight_restricted','height_restricted','multi_linked','road_rating','connect_code'], axis=1)

test = test.drop(['id','base_date', 'road_name', 'start_node_name', 'end_node_name','vehicle_restricted','road_type','weight_restricted','height_restricted','multi_linked','road_rating','connect_code'], axis=1)

print(X_train.shape)
print(y_train.shape)
print(test.shape)

(4701217, 10)
(4701217,)
(291241, 10)


In [16]:
X_train = X_train.replace('월', 0)
X_train = X_train.replace('화', 0)
X_train = X_train.replace('수', 0)
X_train = X_train.replace('목', 0)
X_train = X_train.replace('금', 0)
X_train = X_train.replace('토', 1)
X_train = X_train.replace('일', 1)
test = test.replace('월', 0)
test = test.replace('화', 0)
test = test.replace('수', 0)
test = test.replace('목', 0)
test = test.replace('금', 0)
test = test.replace('토', 1)
test = test.replace('일', 1)

In [17]:
max(X_train['day_of_week'])

1

In [18]:
min(test['day_of_week'])

0

In [19]:
num_1 = 100

In [20]:
X_train['start_latitude'] -= 33
X_train['start_latitude'] = X_train['start_latitude']*num_1

X_train['start_longitude'] -= 126
X_train['start_longitude'] = X_train['start_longitude']*num_1

X_train['end_latitude'] -= 33
X_train['end_latitude'] = X_train['end_latitude']*num_1

X_train['end_longitude'] -= 126
X_train['end_longitude'] = X_train['end_longitude']*num_1

In [21]:
len(test)

291241

In [22]:
test['start_latitude'] -= 33
test['start_latitude'] = test['start_latitude']*num_1

test['start_longitude'] -= 126
test['start_longitude'] = test['start_longitude']*num_1

test['end_latitude'] -= 33
test['end_latitude'] = test['end_latitude']*num_1

test['end_longitude'] -= 126
test['end_longitude'] = test['end_longitude']*num_1

In [23]:
X_train

,day_of_week,base_hour,lane_count,maximum_speed_limit,start_latitude,start_longitude,start_turn_restricted,end_latitude,end_longitude,end_turn_restricted
0,0,17,1,60.0,42.774727,66.261204,0,42.774877,66.233499,0
1,0,21,2,60.0,50.073043,52.910676,1,50.481130,52.624007,0
2,1,7,2,80.0,27.914510,36.859766,0,28.007210,36.214750,0
3,0,13,2,50.0,24.608087,56.720431,0,24.556540,56.622824,0
4,0,8,2,80.0,46.221435,32.655111,0,46.267677,33.015179,0
...,...,...,...,...,...,...,...,...,...,...
4701212,0,16,1,50.0,42.214458,27.812451,0,42.095461,27.375004,0
4701213,0,2,2,80.0,47.250499,42.436822,0,47.252512,42.489016,0
4701214,0,22,2,60.0,44.718271,91.257889,0,44.412080,91.294776,0
4701215,0,2,2,80.0,44.359628,43.181681,0,44.499601,43.333169,0


In [24]:
test

,day_of_week,base_hour,lane_count,maximum_speed_limit,start_latitude,start_longitude,start_turn_restricted,end_latitude,end_longitude,end_turn_restricted
0,0,17,3,70.0,49.942652,54.129817,0,50.077249,54.383661,1
1,0,12,2,70.0,25.850712,42.700345,0,25.811940,41.583951,0
2,0,2,1,60.0,25.895957,47.650760,0,25.920567,47.468740,0
3,0,23,3,70.0,47.349412,54.564685,0,47.106080,54.546676,0
4,0,17,3,70.0,50.147749,56.922319,0,49.686337,58.123010,0
...,...,...,...,...,...,...,...,...,...,...
291236,1,5,1,80.0,47.281887,46.303014,0,47.048337,46.084629,0
291237,0,20,2,60.0,30.535879,59.891375,0,30.079609,60.033182,0
291238,0,11,1,30.0,49.362371,49.676904,1,49.749996,49.694607,1
291239,0,7,2,60.0,25.478224,50.701423,0,25.565928,50.733329,0


LR = lgb.LGBMRegressor(random_state=42).fit(X_train, y_train)

In [25]:
xg_reg = xgb.XGBRegressor(random_state=42).fit(X_train, y_train)

In [26]:
pred = xg_reg.predict(test)

pred = LR.predict(test)

In [27]:
sample_submission = pd.read_csv('C:/Users/82108/JejuRoad/JejuRoadData/open/sample_submission.csv')

In [28]:
sample_submission['target'] = pred
sample_submission.to_csv("C:/Users/82108/JejuRoad/JejuRoadData/open/submit.csv", index = False)

In [29]:
sample_submission

,id,target
0,TEST_000000,24.178860
1,TEST_000001,43.612823
2,TEST_000002,63.212479
3,TEST_000003,35.673599
4,TEST_000004,40.941879
...,...,...
291236,TEST_291236,46.712204
291237,TEST_291237,53.447838
291238,TEST_291238,22.606115
291239,TEST_291239,23.841150
